# Store Sales - Time Series Forecasting
---

Using machine learning to predict grocery sales

In this notebook we will be solving the problem from the competiton: [Store Sales - Time Series Forecasting](https://www.kaggle.com/competitions/store-sales-time-series-forecasting) from [Kaggle](https://www.kaggle.com/)

### Summary:
*   In this competition, you’ll use time-series forecasting to forecast store sales on data from **Corporación Favorita**, a large Ecuadorian-based grocery retailer.
    *   Specifically, you'll build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores.  
    You'll practice your machine learning skills with an approachable training dataset of dates, store, and item information, promotions, and unit sales.

* The evaluation metric for this competition is Root Mean Squared Logarithmic Error - RMSLE.

### File Descriptions and Data Field Information

#### `train.csv`
The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.

*   **store_nbr** identifies the store at which the products are sold.
*   **family** identifies the type of product sold.
*   **sales** gives the total sales for a product family at a particular store at a given date.  
    Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
*   **onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date.

#### `test.csv`
The test data, having the same features as the training data.  
You will predict the target sales for the dates in this file.  
*   The dates in the test data are for the 15 days after the last date in the training data.

#### `sample_submission.csv`
*   A sample submission file in the correct format.

#### `stores.csv`
Store metadata, including city, state, type, and cluster.
*   **cluster** is a grouping of similar stores.

#### `oil.csv`
Daily oil price.  
Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

#### `holidays_events.csv`
Holidays and Events, with metadata  


### NOTES:
Pay special attention to the transferred column.  
*   A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government.

*   A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is **Transfer**.  
    *   For example, the holiday ***Independencia de Guayaquil*** was transferred from **2012-10-09** to **2012-10-12**, which means it was celebrated on **2012-10-12**.  

*   Days that are type **Bridge** are extra days that are added to a holiday (e.g., to extend the break across a long weekend).  

*   These are frequently made up by the type **Work Day** which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the **Bridge**.

*   Additional holidays are days added a regular calendar holiday, for example, as typically happens around **Christmas** (making Christmas Eve a holiday).

*   Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. **Supermarket sales could be affected by this.**

*   A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

# Importing Libraries:
---

To keep the code more concise and undestandable, here I will be listing all of the libraries used during the code

In [67]:
# System libraries
import sys
import zipfile
import pathlib
import os
import dotenv

# Data Manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px

# Statistical Analysis
from statsmodels.tsa.stattools import adfuller
from scipy.stats import ttest_ind
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

# Modelling
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import RandomizedSearchCV
import pickle

# Warning filter
import warnings
warnings.filterwarnings("ignore")

## Loading Data:

In [2]:
pathlib.Path("data").mkdir(parents=True, exist_ok=True)

file_path = pathlib.Path("data/train.csv")
if not file_path.is_file():
    with zipfile.ZipFile("./store-sales-time-series-forecasting.zip", 'r') as zf:
        zf.extractall("./data/")

## Reading all files:

In [80]:
# Assign all files to a variable:

# sales
sales = pd.read_csv('./data/train.csv', dtype={'date':'str', 'store_nbr':'category'}, parse_dates=['date'])

# stores
stores = pd.read_csv('./data/stores.csv', dtype={'store_nbr':'category', 'cluster':'category'})

# oil prices
oil = pd.read_csv('./data/oil.csv', dtype={'date':'str'}, parse_dates=['date']).rename(columns={'dcoilwtico':'oil_price'})

# holiday events
holidays = pd.read_csv('./data/holidays_events.csv', dtype={'date':'str'}, parse_dates=['date'])

# transactions
transactions = pd.read_csv('./data/transactions.csv', dtype={'date':'str', 'store_nbr':'category'}, parse_dates=['date'])

# train set
train = sales.copy()

# test set
test = pd.read_csv('./data/test.csv', dtype={'date':'str'}, parse_dates=['date'])

# Exploratory Data Analysis (EDA)
---

In [15]:
# printing the head of each file
files = {'sales':sales,
         'stores':stores,
         'oil':oil,
         'holidays':holidays,
         'transactions':transactions}

for filename, data in files.items():
    print(f'FILE: {filename}')
    display(data.head())
    print()

FILE: sales


,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0



FILE: stores


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4



FILE: oil


,date,oil_price
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20



FILE: holidays


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False



FILE: transactions


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


## 1. Undestanding the Datasets

### 1.1 Shape of the datasets
---

In [18]:
# train
print('Train:', train.shape)

# test
print('Test:', test.shape, end='\n\n')

# stores
print('Stores:', stores.shape)

# transactions
print('Transactions:', transactions.shape)

# holidays
print('Holidays:', holidays.shape)

# oil price
print('Oil:', oil.shape)

Train: (3000888, 6)
Test: (28512, 5)

Stores: (54, 5)
Transactions: (83488, 3)
Holidays: (350, 6)
Oil: (1218, 2)


The train dataset contains 3,000,888 (99.06% of instances) rows and 6 columns while the test dataset contains 28,512 (00.94% of instances) rows and 5 columns.  


The train dataset is **significantly** larger than the test dataset in terms of the number of rows. This is expected (specially in forecasting problems) as the train dataset is usually larger to provide sufficient data for model training.

### 1.2 Features (columns) Information
---

In [27]:
def show_features_info(name:str, df:pd.DataFrame):
    print(f'Features Informations for: {name}', end='\n\n')
    df.info()

In [30]:
dataframes = {'train':train,
              'stores':stores,
              'oil':oil,
              'holidays':holidays,
              'transactions':transactions,
              'test':test}

for name, df in dataframes.items():
    show_features_info(name, df)
    print(end='\n\n')

Features Informations for: train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


Features Informations for: stores

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


Features Informations for: oil

<class 'pan

The train dataset contains **3,000,888** entries and 6 columns: 'id', 'date', 'store_nbr', 'family', 'sales', and 'onpromotion'.

The test dataset contains **28,512** entries and 5 columns: 'id', 'date', 'store_nbr', 'family', and 'onpromotion'.

As expected, the test dataset does not have the "**sales**" column. This column is not needed because 'sales' is the variable we want to predict. The goal is to use the trained model to predict or forecast the sales in the test data based on the other available features.

*   The Holiday Events Dataset:  
The dataset contains 350 entries and 6 columns: 'date', 'type', 'locale', 'locale_name', 'description', and 'transferred'.
The "date" column in the dataset is of type object. It needs to be converted to a datetime data type for further analysis.


*   The Oil Dataset:  
The dataset contains 1,218 entries has 2 columns: 'date' and 'dcoilwtico'.
The "date" column in the dataset is of type object. It needs to be converted to a datetime data type for further analysis.
The 'dcoilwtico' column has 1,175 non-null values, indicating that there are some missing values in this column.


*   The Stores dataset:  
The dataset contains 54 entries and 5 columns: 'store_nbr', 'city', 'state', 'type', and 'cluster'.


*   The Transactions dataset:  
The dataset contains 83,488 entries and 3 columns: 'date', 'store_nbr', and 'transactions'.
The "date" column in the dataset is of type object. It needs to be converted to a datetime data type for further analysis.

Since we did `pd.read_csv(..., dtype={'date':'str'}, parse_dates=['date'], ...)`, we already have the 'date' column in all datasets with the same type: `datetime64[ns]`

### 1.3 Summary Statistics
---

In [57]:
def summary_statistics(name:str, df:pd.DataFrame):
    if df.select_dtypes(include=[np.int64, np.int32, np.float64, np.float32]).shape[1] > 0:
        print(f'Features Statistics for Numerical Data in: {name}', end='\n\n')
        display(df.select_dtypes(include=[np.int64, np.int32, np.float64, np.float32]).describe())

In [58]:
dataframes = {'train':train,
              'stores':stores,
              'oil':oil,
              'holidays':holidays,
              'transactions':transactions}

for name, df in dataframes.items():
    summary_statistics(name, df)
    print(end='\n\n')

Features Statistics for Numerical Data in: train



,id,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,3.577757e+02,2.602770e+00
std,8.662819e+05,1.101998e+03,1.221888e+01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,0.000000e+00,0.000000e+00
50%,1.500444e+06,1.100000e+01,0.000000e+00
75%,2.250665e+06,1.958473e+02,0.000000e+00
max,3.000887e+06,1.247170e+05,7.410000e+02






Features Statistics for Numerical Data in: oil



,oil_price
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000






Features Statistics for Numerical Data in: transactions



,transactions
count,83488.000000
mean,1694.602158
std,963.286644
min,5.000000
25%,1046.000000
50%,1393.000000
75%,2079.000000
max,8359.000000


As you can see, the **stores** and **holidays** datasets don't have numerical data:

* Stores:

    |Column         |Type          |
    |---------------|--------------|
    |store_nbr      |object        |
    |city           |object        |
    |state          |object        |        
    |type           |object        |        
    |cluster        |object        |

<br>

* Holidays:

    |Column         |Type          |
    |---------------|--------------|
    |date           |datetime64[ns]|
    |type           |object        |
    |locale         |object        |        
    |locale_name    |object        |        
    |description    |object        |      
    |transferred    |bool          |

For the **test** set, we are not going to summarize it, since we weren't suppose to look at it!

### 1.4 Missing Values
---

In [65]:
def show_missing(name:str, df:pd.DataFrame|pd.Series):
    print(f'Missing data for: {name}', end='\n')
    display(df.isnull().sum())
    print("===" * 10)

In [66]:
dataframes = {'train':train,
              'stores':stores,
              'oil':oil,
              'holidays':holidays,
              'transactions':transactions,
              'test':test}

for name, df in dataframes.items():
    show_missing(name, df)
    print(end='\n\n')

Missing data for: train


id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64



Missing data for: stores


store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64



Missing data for: oil


...

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64



Missing data for: transactions


date            0
store_nbr       0
transactions    0
dtype: int64



Missing data for: test


id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64

As we can see, the only dataframe that has missing data is the `oil`, so let's focus in this dataset.

In [81]:
# Visualizing the 'dcoilwtico' column to Identify a Strategy for Handling Missing Values
fig = px.line(oil, x='date', y='oil_price')
fig.update_layout(title='Trend of Oil Prices Over Time', title_x=0.5, xaxis_title='Date', yaxis_title='Oil Price')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

As shown above, the missing data occurs randomly on specific days, not on consecutive days. In this case we will use the backfill strategy.

In [86]:
oil['oil_price'] = oil['oil_price'].bfill()
oil.isnull().sum()

date         0
oil_price    0
dtype: int64